## ETAPA 1

#### Cargar datos ☑︎
#### Cargar modelos ☒
#### Entrenar modelos ☒
#### Ver resultados ☒

In [2]:
import glob
import shutil
import os
import re
import random

In [122]:
# BLOQUE 1: Separar data-set en carpetas

#file = open("Data-set/PA-expert-annotations.txt", "r")
#os.mkdir("Data-set/datos")
#os.mkdir("Data-set/datos/01-Normal")
#os.mkdir("Data-set/datos/02-Tapered")
#os.mkdir("Data-set/datos/03-Pyriform")
#os.mkdir("Data-set/datos/04-Small")
#os.mkdir("Data-set/datos/05-Amorphous")

#categorias = ["01-Normal", "02-Tapered", "03-Pyriform", "04-Small", "05-Amorphous"]

#for x in file:
#    # Se obtiene el nombre y clase desde el archivo txt
#    aux1 = x.split('	')
#    clase = int(aux1[4].replace('\n',''))
#    array.append(clase)
#    aux2 = aux1[0].replace('\n','').split('-')
#    p = aux2[0]
#    pl = aux2[1]
#    aux3 = aux2[2].split('/')
#    n_sample = int(re.split('(\d+)',aux3[0])[1])
#    n_sperm = int(re.split('(\d+)',aux3[1])[1])
#    # Se conforma el directorio de la imagen a partir de la informacion anterior.
#    file = 'Data-set/Partial-Agreement-Images/ch00_'+p+'-'+pl+'-sample'+str(n_sample)+'-sperm'+str(n_sperm)+'.tif'
#    # Se conforma el directorio donde se va a copiar la imagen
#    dest_dir = 'Data-set/data/'
#    if (clase == 0):
#        dest_dir=dest_dir+'01-Normal'
#    elif (clase == 1):
#        dest_dir=dest_dir+'02-Tapered'
#    elif (clase == 2):
#        dest_dir=dest_dir+'03-Pyriform'
#    elif (clase == 3):
#        dest_dir=dest_dir+'04-Small'
#    else:
#        dest_dir=dest_dir+'05-Amorphous'
#    
#    # Se copia la imagen
#    shutil.copy(file,dest_dir)

In [7]:
def leerClases( dir ):
    file = open(dir, "r")
    clases = []
    for x in file:
        # Se obtiene el nombre y clase desde el archivo txt
        aux1 = x.split('	')
        clase = int(aux1[4].replace('\n',''))
        clases.append(clase) 
    return clases


In [125]:
# BLOQUE 2: Crear data-set balanceado

# Se aplica la estrategia: Random subsampling en clase mayoritaria AMORFO

clases = leerClases("Data-set/PA-expert-annotations.txt")
print("Clase 0: "+str(clases.count(0))+"\n"+"Clase 1: "+str(clases.count(1))+
      "\n"+"Clase 2: "+str(clases.count(2))+"\n"+"Clase 3: "+str(clases.count(3))+
      "\n"+"Clase 4: "+str(clases.count(5))+"\n")

# Se obtiene promedio entre las clases minotarias ( Clase 0, 1, 2 y 3)
promedio = (clases.count(0)+clases.count(1)+clases.count(2)+clases.count(3))/4


lista = os.listdir('Data-set/b_data/05-Amorphous')

# Se eliminan aleatoriamentes elementos de la clase mayoritaria hasta llegar al promedio.
while len(lista) > promedio:
    elemento = random.choice(lista)
    lista.remove(elemento)
    elemento = 'Data-set/b_data/05-Amorphous/'+elemento
    os.remove( elemento )


Clase 0: 100
Clase 1: 228
Clase 2: 76
Clase 3: 72
Clase 4: 656



In [5]:
# Dividir Data-set original y balanceado.

def crearSubConjuntos( nombre_clase, dir_clase, dest_dir):
    os.mkdir(dest_dir+"/train/"+nombre_clase)
    os.mkdir(dest_dir+"/test/"+nombre_clase)
    os.mkdir(dest_dir+"/valid/"+nombre_clase)
    
    lista = os.listdir(dir_clase)
    contador = 0
    while (len(lista)*0.1) > contador:
        elemento = random.choice(lista)
        lista.remove(elemento)
        elemento = dir_clase+'/'+elemento
        shutil.copy(elemento,dest_dir+"/test/"+nombre_clase)
        contador = contador+1

    contador = 0
    while (len(lista)*0.2) > contador:
        elemento = random.choice(lista)
        lista.remove(elemento)
        elemento = dir_clase+'/'+elemento
        shutil.copy(elemento,dest_dir+"/valid/"+nombre_clase)
        contador = contador+1

    while len(lista) > 0:
        elemento = random.choice(lista)
        lista.remove(elemento)
        elemento = dir_clase+'/'+elemento
        shutil.copy(elemento,dest_dir+"/train/"+nombre_clase)
        contador = contador+1

def crearConjuntos( root_dir, dest_dir ):
    os.mkdir(dest_dir+"/train")
    os.mkdir(dest_dir+"/test")
    os.mkdir(dest_dir+"/valid")
    crearSubConjuntos( '01-Normal', root_dir+'/01-Normal', dest_dir )
    crearSubConjuntos( '02-Tapered', root_dir+'/02-Tapered', dest_dir )
    crearSubConjuntos( '03-Pyriform', root_dir+'/03-Pyriform', dest_dir )
    crearSubConjuntos( '04-Small', root_dir+'/04-Small', dest_dir )
    crearSubConjuntos( '05-Amorphous', root_dir+'/05-Amorphous', dest_dir )

